The purpose of this code is to take batches of simulation data and plot them on a map. It does some calculations also to determine circular error probable, which helps bound likely impact points.

A valid Google API key must be entered for plotting capability. Be aware of not publishing it to the internet.

In [ ]:
%run Trajectory_Simulation.ipynb
import pandas as pd

from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import gmap, figure
from bokeh.models import (
    Ellipse, Annulus,
    GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    PanTool, WheelZoomTool, BoxSelectTool, Range1d, Arrow, Segment, Ray
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5, Inferno256

API_KEY = ''

In [ ]:
# linearization of globe based on our latitude
# this is for calculating displacements from (0, 0, 0) in m as displacements in deg
# given the size of Earth, error from linearization should be negligible
# refer to https://gis.stackexchange.com/questions/75528/understanding-terms-in-length-of-degree-formula
def m_per_deg_lat(lat):
    lat = np.radians(lat)
    return 111132.92 - 559.82*np.cos(2*lat) + 1.175*np.cos(4*lat) - 0.0023*np.cos(6*lat)

def m_per_deg_long(long):
    long = np.radians(long)
    return 111412.84 - 93.5*cos(3*long) + 0.118*cos(5*long)

def lat_long(df):
    array = df.loc[:, ['x', 'y', 'z']]
    X = [x for x in array['x']]
    Y = [y for y in array['y']]
    Z = [z for z in array['z']]
    coords = [Environment(None, 1).ECEF_to_geodetic([X[i], Y[i], Z[i]])
                                   for i in range(len(array))]
    df['lat'] = [coord[0] for coord in coords]
    df['long'] = [coord[1] for coord in coords]
    df['height'] = [coord[2] for coord in coords]

# mean impact point
def get_mean_cartesian(df):
    mean_x, mean_y, mean_z = df.loc[:, ['x', 'y', 'z']].describe().loc['mean']
    return mean_x, mean_y, mean_z

# mean impact point
def get_mean_geode(df):
    mean_lat, mean_long = df.loc[:, ['lat', 'long']].describe().loc['mean']
    return mean_lat, mean_long

In [ ]:
def calc_CEP(results_df, mean, target):
    #dx = m_per_deg_long(mean[1])
    #dy = m_per_deg_lat(mean[0])
    # CEP = sqrt(MSE) = sqrt(GPS_error + bias + var_norm + var_theta + cov_norm_theta)
    # this is the mean distance from the mean impact point
    variance = results_df.loc[:,['lat','long']].var(axis=0).sum()
    covariance = results_df.loc[:,['lat','long']].cov().loc['lat','long']
    # GPS is 95% inside a 2x2 m box, and 95% of values on one axis are within 2*RMS of ideal
    # thus RMS = 1/2, since 2* 1/2 =1, and 1 on each side gives a 2 m bound. Therefore MSE = 1/4, which we double
    # because there are two axes to account for
    #GPS_MSE = 1/4
    diff = np.array(target) - np.array(mean)
    #diff[0] *= dy
    #diff[1] *= dx
    bias = np.linalg.norm(diff)
    total_MSE = variance + covariance + bias #+ 2*GPS_MSE
    CEP = np.sqrt(total_MSE)
    return CEP

# this requires a google api key. ask me on slack if you want to use mine, please keep it secret, keep it safe
def plot_on_map(df, nominal, launch_site):#, release_pt, mean_wind, release_theta, CEP):
    dX = m_per_deg_long(nominal[1])
    dY = m_per_deg_lat(nominal[0])
        
    mean_lat, mean_long = get_mean_geode(df)
    CEP = calc_CEP(df, [mean_lat, mean_long], nominal)
    
    dy = CEP
    dx = CEP
    
    map_options = GMapOptions(lat=mean_lat, lng=mean_long, map_type="terrain", zoom=6, scale_control=True)
    #plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
    plot = gmap(google_api_key = API_KEY, map_options=map_options)
    plot.title.text = "Impact Points"
    plot.api_key = API_KEY
    plot.match_aspect=True
    
    source = ColumnDataSource(
    data=dict(
        lat=df.lat.tolist(),
        lon=df.long.tolist()))
    
    #color_mapper = LinearColorMapper(palette=Inferno256, low=min(df['v_w']), high=max(df['v_w']))
    nominal_pt = Circle(x=nominal[1], y=nominal[0], size=10, fill_color='blue',
                    fill_alpha=0.5, line_color=None)
    mean_pt = Circle(x=mean_long, y=mean_lat, size=10, fill_color='purple',
                    fill_alpha=0.5, line_color=None)
    launch_pt = Circle(x=launch_site[1], y=launch_site[0], size=10, fill_color='green',
                    fill_alpha=0.5, line_color=None)
    circle = Circle(x="lon", y="lat", size=5, fill_color='red',
                    fill_alpha=0.5, line_color=None)
    
    plot.add_glyph(source, circle)
    plot.add_glyph(nominal_pt)
    plot.add_glyph(mean_pt)
    plot.add_glyph(launch_pt)
    
    plot.arc(x=nominal[1], y=nominal[0], radius=CEP*dY, start_angle=0, end_angle=2*np.pi, line_color='red')
    plot.arc(x=nominal[1], y=nominal[0], radius=CEP*2*dY, start_angle=0, end_angle=2*np.pi, line_color='green')
    plot.arc(x=nominal[1], y=nominal[0], radius=CEP*3*dY, start_angle=0, end_angle=2*np.pi, line_color='blue')
    
    output_notebook()

    show(plot)

Note that if the data is in multiple files, it must be concatenated. Also, when a table is loaded from a file, the first column is just indices, so we must drop it.

In [ ]:
sim_data = pd.concat(map(pd.read_csv, ['./dispersion_sample_data/sim_data1.csv',
                                        './dispersion_sample_data/sim_data2.csv',
                                        './dispersion_sample_data/sim_data3.csv',
                                        './dispersion_sample_data/sim_data4.csv',
                                       './dispersion_sample_data/sim_data5.csv']))
sim_data.drop('Unnamed: 0', axis='columns', inplace=True)
lat_long(sim_data)
print(sim_data.describe())

In [ ]:
plot_on_map(sim_data, nominal=[34.58299979, -106.98085475], launch_site=[32.9895472, -106.9694681])